In [52]:
import logging, time, argparse, random
from Base import Base
from LogisticRegression import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
l = 0.3

logging.basicConfig(filename='resultslogistic.log', level=logging.INFO)
logging.info('Starting Logistic Regression proccess ith lambda {}'.format(l))

b = Base("../assignment1_2016S1/training_data.csv",
         "../assignment1_2016S1/training_labels.csv",
         "../assignment1_2016S1/test_data.csv")
## load data
start_time = time.time()
logging.info('Loading data')
X_train, y_train, X_test, test_names = b.load_data()
logging.info("--- %s seconds ---" % (time.time() - start_time))
m, n = X_train.shape

# PCA

In [3]:
start_time = time.time()
time_pca = 0
logging.info('Reduce matrix using PCA')
X_train_pca = b.dimension_reduction(X_train, "pca",0.9)
m,n = X_train.shape
X_test_pca = b.dimension_reduction(X_test, "pca",0.90)
time_pca = (time.time() - start_time)
time_pca

612.0232906341553

# Logistic Regression Algorithm

In [4]:
clf = LogisticRegression()

Logistic Regression Class created


## Cross Validation without PCA

In [6]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_log_no_pca = 0

k = 10
results_log_cross_no_pca = []
expected_log_cross_no_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train[training], y_train_cross, l)
    y_pred = clf.predict(X_train[validation])

    ## append results
    results_log_cross_no_pca.append(y_pred)
    expected_log_cross_no_pca.append(y_val_cross)
    print("Accuracy: "+str(clf.score(y_pred,y_val_cross)))
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_log_no_pca = (time.time() - start_time)
time_cross_log_no_pca

cross validation iteration 0
Accuracy: 0.6628543013426156
cross validation iteration 1
Accuracy: 0.6673296867230234
cross validation iteration 2
Accuracy: 0.6464445549477872
cross validation iteration 3
Accuracy: 0.6747886623570363
cross validation iteration 4
Accuracy: 0.6592039800995025
cross validation iteration 5
Accuracy: 0.6666666666666666
cross validation iteration 6
Accuracy: 0.6621890547263681
cross validation iteration 7
Accuracy: 0.6711442786069651
cross validation iteration 8
Accuracy: 0.6308457711442786
cross validation iteration 9
Accuracy: 0.6467661691542289


1953.0766277313232

In [27]:
precall_res_log_no_pca = []
for res,exp in zip(results_log_cross_no_pca,expected_log_cross_no_pca):
    precall_res_log_no_pca.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_log_no_pca,10)

(0.65620585009019194,
 0.012811885373577532,
 0.66371565885132233,
 0.01308581930956914,
 0.65374331867532842,
 0.012864650319925985)

## Cross Validation with PCA

In [7]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_log_pca = 0

k = 10
results_log_cross_pca = []
expected_log_cross_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train_pca[training], y_train_cross, l)
    y_pred = clf.predict(X_train_pca[validation])

    ## append results
    results_log_cross_pca.append(y_pred)
    expected_log_cross_pca.append(y_val_cross)
    print("Accuracy: "+str(clf.score(y_pred,y_val_cross)))
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_log_pca = (time.time() - start_time)
time_cross_log_pca

cross validation iteration 0
Accuracy: 0.6111387369467927
cross validation iteration 1
Accuracy: 0.6330183988065639
cross validation iteration 2
Accuracy: 0.6220785678766783
cross validation iteration 3
Accuracy: 0.6141223272003978
cross validation iteration 4
Accuracy: 0.6169154228855721
cross validation iteration 5
Accuracy: 0.6144278606965174
cross validation iteration 6
Accuracy: 0.608955223880597
cross validation iteration 7
Accuracy: 0.6044776119402985
cross validation iteration 8
Accuracy: 0.6203980099502487
cross validation iteration 9
Accuracy: 0.6263681592039801


2266.1346893310547

In [29]:
precall_res_log_pca = []
for res,exp in zip(results_log_cross_pca,expected_log_cross_pca):
    precall_res_log_pca.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_log_pca,10)

(0.61416249481135499,
 0.0075669990767129845,
 0.60804650838425367,
 0.007510505134974536,
 0.59987324371955886,
 0.007468705041030205)

## 2/3 - 1/3 Split

In [10]:
logging.info('Training in 2/3 of data and testing in 1/3 of data')
start_time = time.time()
result_log_split = []
time_log_split_no_pca = 0

## split the dataset
items = list(range(m))
random.shuffle(items)
training = items[m // 3:]
testing = items[:m // 3]

## list of labels in training and testing
y_val = [y_train[y] for y in training]
y_test = [y_train[y] for y in testing]

## train and predict in testing data
clf.fit(X_train[training], y_val, l)
result_log_split = clf.predict(X_train[testing])

time_log_split_no_pca = (time.time() - start_time)
## confusion matrix
#b.plot_confusion_matrix(y_test, result_split_no_pca, clf.list_classes)
time_log_split_no_pca

140.74104189872742

In [16]:
clf.score(result_log_split,y_test)

0.6494553051783316

## 2/3 - 1/3 Split PCA

In [18]:
logging.info('Training in 2/3 of data and testing in 1/3 of data')
start_time = time.time()
result_log_split_pca = []
time_log_split_pca = 0

## split the dataset
items = list(range(m))
random.shuffle(items)
training = items[m // 3:]
testing = items[:m // 3]

## list of labels in training and testing
y_val_log_pca = [y_train[y] for y in training]
y_test_log_pca = [y_train[y] for y in testing]

## train and predict in testing data
clf.fit(X_train_pca[training], y_val_log_pca, l)
result_log_split_pca = clf.predict(X_train_pca[testing])

time_log_split_pca = (time.time() - start_time)
## confusion matrix
#b.plot_confusion_matrix(y_test, result_split_pca, clf.list_classes)
time_log_split_pca

148.41272068023682

In [19]:
clf.score(y_test_log_pca,result_log_split_pca)

0.6196090135800627

## Test Set

In [ ]:
start_time = time.time()
time_log_test_set = 0
result_log_test = []
## train and predict in testing data
clf.fit(X_train, y_train, lmda)
result_log_test = clf.predict(X_test)
results = zip(test_names,result_log_test)
b.save_data(results,'test_data.csv')
time_log_test_set = (time.time() - start_time)
time_log_test_set

## Test Set PCA

In [17]:
start_time = time.time()
time_log_test_set_pca = 0
result_log_test_pca = []
## train and predict in testing data
clf.fit(X_train_pca, y_train, lmda)
result_log_test_pca = clf.predict(X_test_pca)
results = zip(test_names,result_log_test_pca)
b.save_data(results,'test_data.csv')
time_log_test_set_pca = (time.time() - start_time)
time_log_test_set_pca

NameError: name 'lmda' is not defined

# Gaussian Naive Bayes

In [31]:
clf = GaussianNB()

## Cross Validation without PCA

In [34]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_gnb_no_pca = 0

k = 10
results_gnb_cross_no_pca = []
expected_gnb_cross_no_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train[training], y_train_cross)
    y_pred = clf.predict(X_train[validation])

    ## append results
    results_gnb_cross_no_pca.append(y_pred)
    expected_gnb_cross_no_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_gnb_no_pca = (time.time() - start_time)
time_cross_gnb_no_pca

cross validation iteration 0
cross validation iteration 1
cross validation iteration 2
cross validation iteration 3
cross validation iteration 4
cross validation iteration 5
cross validation iteration 6
cross validation iteration 7
cross validation iteration 8
cross validation iteration 9


99.09266424179077

In [35]:
precall_res_gnb = []
for res,exp in zip(results_gnb_cross_no_pca,expected_gnb_cross_no_pca):
    precall_res_gnb.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_gnb,10)

(0.44338979104758414,
 0.011183814237737738,
 0.45091058637052972,
 0.0125824423440414,
 0.44290864017472265,
 0.011339273950491)

## Cross Validation with PCA

In [38]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_gnb_pca = 0

k = 10
results_gnb_cross_pca = []
expected_gnb_cross_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train_pca[training], y_train_cross)
    y_pred = clf.predict(X_train_pca[validation])

    ## append results
    results_gnb_cross_pca.append(y_pred)
    expected_gnb_cross_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_gnb_pca = (time.time() - start_time)
time_cross_gnb_pca

cross validation iteration 0
cross validation iteration 1
cross validation iteration 2
cross validation iteration 3
cross validation iteration 4
cross validation iteration 5
cross validation iteration 6
cross validation iteration 7
cross validation iteration 8
cross validation iteration 9


31.824105501174927

In [40]:
precall_res_gnb_pca = []
for res,exp in zip(results_gnb_cross_pca,expected_gnb_cross_pca):
    precall_res_gnb_pca.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_gnb_pca,10)

(0.14903055008739466,
 0.010761382435128756,
 0.31356060823863535,
 0.017264945571249014,
 0.15383318503157889,
 0.010485893075177834)

# Multinomial Naive Bayes

## Cross Validation without PCA

In [43]:
clf = MultinomialNB()

In [44]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_mnb_no_pca = 0

k = 10
results_mnb_cross_no_pca = []
expected_mnb_cross_no_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train[training], y_train_cross)
    y_pred = clf.predict(X_train[validation])

    ## append results
    results_mnb_cross_no_pca.append(y_pred)
    expected_mnb_cross_no_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_mnb_no_pca = (time.time() - start_time)
time_cross_mnb_no_pca

cross validation iteration 0
cross validation iteration 1
cross validation iteration 2
cross validation iteration 3
cross validation iteration 4
cross validation iteration 5
cross validation iteration 6
cross validation iteration 7
cross validation iteration 8
cross validation iteration 9


17.00609016418457

In [45]:
precall_res_mnb = []
for res,exp in zip(results_mnb_cross_no_pca,expected_mnb_cross_no_pca):
    precall_res_mnb.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_mnb,10)

(0.6194129684931069,
 0.007275881532495411,
 0.65703534531866004,
 0.009344078543092724,
 0.61142729803667206,
 0.007568187928998158)

## Cross Validation with PCA

In [46]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_mnb_pca = 0

k = 10
results_mnb_cross_pca = []
expected_mnb_cross_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train_pca[training], y_train_cross)
    y_pred = clf.predict(X_train_pca[validation])

    ## append results
    results_mnb_cross_pca.append(y_pred)
    expected_mnb_cross_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_mnb_pca = (time.time() - start_time)
time_cross_mnb_pca

cross validation iteration 0


ValueError: Input X must be non-negative

# Nearest Neighbors 

In [53]:
clf = KNeighborsClassifier(n_neighbors=6)

## Cross Validation without PCA

In [55]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_knn_no_pca = 0

k = 10
results_knn_cross_no_pca = []
expected_knn_cross_no_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train[training], y_train_cross)
    y_pred = clf.predict(X_train[validation])

    ## append results
    results_knn_cross_no_pca.append(y_pred)
    expected_knn_cross_no_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_knn_no_pca = (time.time() - start_time)
time_cross_knn_no_pca

cross validation iteration 0
cross validation iteration 1
cross validation iteration 2
cross validation iteration 3
cross validation iteration 4
cross validation iteration 5
cross validation iteration 6
cross validation iteration 7
cross validation iteration 8
cross validation iteration 9


7091.431382417679

In [56]:
precall_res_knn = []
for res,exp in zip(results_knn_cross_no_pca,expected_knn_cross_no_pca):
    precall_res_knn.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_knn,10)

(0.33576267394150955,
 0.016744730117966754,
 0.55462278966029999,
 0.024707902532493962,
 0.3807205448625644,
 0.019555245004247837)

In [57]:
for res,exp in zip(results_knn_cross_no_pca,expected_knn_cross_no_pca):
    print(clf.score(res,exp))

C:\Users\rafae\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: could not convert string to float: 'Racing'

In [58]:
logging.info('Starting 10-fold cross validation')
start_time = time.time()
time_cross_knn_pca = 0

k = 10
results_knn_cross_pca = []
expected_knn_cross_pca = []
c_cross = 0

## call training and validation indexes
for training, validation in b.cross_validation(k, m):
    print('cross validation iteration {}'.format(c_cross))

    ## list of labels in training and validation
    y_train_cross = [y_train[y] for y in training]
    y_val_cross = [y_train[y] for y in validation]

    ## train and predict in validation data
    clf.fit(X_train_pca[training], y_train_cross)
    y_pred = clf.predict(X_train_pca[validation])

    ## append results
    results_knn_cross_pca.append(y_pred)
    expected_knn_cross_pca.append(y_val_cross)
    
    c_cross += 1

## calculate and store averaged results and duration
time_cross_mnb_pca = (time.time() - start_time)
time_cross_mnb_pca

cross validation iteration 0
cross validation iteration 1
cross validation iteration 2
cross validation iteration 3
cross validation iteration 4
cross validation iteration 5
cross validation iteration 6
cross validation iteration 7
cross validation iteration 8
cross validation iteration 9


2012.990224123001

In [60]:
precall_res_knn_pca = []
for res,exp in zip(results_knn_cross_pca,expected_knn_cross_pca):
    precall_res_knn_pca.append(precision_recall_fscore_support(res, exp, average='macro'))
b.get_precision_recall_fscore_overall(precall_res_knn_pca,10)

(0.39685868832311472,
 0.010353264057966135,
 0.56005632316017429,
 0.013473237679498797,
 0.43579173885310957,
 0.008296883607966543)